# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV


# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(132027, 17)
(33007, 17)
(132027,)
(33007,)


In [ ]:
rfc = RandomForestClassifier(criterion='entropy', random_state=42)
model = rfc.fit(X_train, y_train)
importance = model.feature_importances_
tabel = X_df.columns
for f, fi in zip(tabel, importance):
    print('Feature : %24s, score %.5f' %(f, fi))

Feature :                       id, score 0.08041
Feature :               CustomerId, score 0.07920
Feature :              CreditScore, score 0.07684
Feature :                      Age, score 0.17311
Feature :                   Tenure, score 0.04516
Feature :                  Balance, score 0.07086
Feature :            NumOfProducts, score 0.13214
Feature :                HasCrCard, score 0.01063
Feature :           IsActiveMember, score 0.04004
Feature :          EstimatedSalary, score 0.07895
Feature :                   France, score 0.00762
Feature :                  Germany, score 0.01994
Feature :                    Spain, score 0.00573
Feature :                   Female, score 0.00894
Feature :                     Male, score 0.00801
Feature :         ExitedfamilySize, score 0.08047
Feature :      NotExitedfamilySize, score 0.08194


In [ ]:
X1 = X_df.drop(columns=['Female','Male','Germany','Spain','France','HasCrCard']).values
X_train, X_test, y_train, y_test = train_test_split(X1, y_df, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(132027, 11)
(33007, 11)
(132027,)
(33007,)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi scaler
scaler = MinMaxScaler()
# Fit scaler pada data dengan 10 fitur
scaler.fit(X_train)  # Asumsi X_train_10_features adalah data yang sudah di-drop kolom

# Gunakan scaler yang sudah di-fit untuk transform data lain yang memiliki 10 fitur yang sama
X1_scaled = scaler.transform(X1)
X_submission_10_features = X_submission.drop(columns=['Female','Male','Germany','Spain','France','ExitedfamilySize'])

# Transform X_submission dengan scaler yang sudah di-fit
X_submission_scaled = scaler.transform(X_submission_10_features)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


# Model Training

In [ ]:
import numpy as np

n_estimators = list(np.linspace(200, 2000, num=100, dtype=np.int32))
max_features = ['auto', 'sqrt']
 
max_depth = list(np.linspace(10, 110, num=11, dtype=np.int32))
max_depth.append(None)
 
min_samples_split = [2, 5, 10]
 
min_samples_leaf = [1, 2, 4]
 
bootstrap = [True, False]
 
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#Random Forest
rfc = RandomForestClassifier(criterion='entropy', random_state=42)

rf_random = RandomizedSearchCV(estimator=rfc, param_distributions=random_grid, n_iter=10, cv=3, verbose=2, random_state=0)

#Fit the random search model
rf_random.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=781; total time= 6.7min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=781; total time= 6.7min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=781; total time= 8.3min
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=436; total time= 9.6min
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=436; total time= 9.8min
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=436; total time= 9.7min
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# Evaluation on the test set
y_pred = rf_random.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
rf_random.fit(X1_scaled, y_df)

In [ ]:
# GridSearchCV setup
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best parameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 500}
ROC-AUC on test set: 0.886015807791595


In [ ]:
# Evaluation on the test set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X1, y_df)

              precision    recall  f1-score   support

           0       0.88      0.95      0.92     26052
           1       0.75      0.53      0.62      6955

    accuracy                           0.86     33007
   macro avg       0.81      0.74      0.77     33007
weighted avg       0.85      0.86      0.85     33007

Confusion Matrix:
[[24799  1253]
 [ 3268  3687]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [20],
                         'classifier__min_samples_leaf': [4],
                         'classifier__min_samples_split': [10],
                         'classifier__n_estimators': [500]},
             scoring='roc_auc', verbose=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi scaler
scaler = MinMaxScaler()
# Fit scaler pada data dengan 10 fitur
scaler.fit(X_train)  # Asumsi X_train_10_features adalah data yang sudah di-drop kolom

# Gunakan scaler yang sudah di-fit untuk transform data lain yang memiliki 10 fitur yang sama
X1_scaled = scaler.transform(X1)
X_submission_10_features = X_submission.drop(columns=['Female','Male','Germany','Spain','France','ExitedfamilySize'])

# Transform X_submission dengan scaler yang sudah di-fit
X_submission_scaled = scaler.transform(X_submission_10_features)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission_10_features)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission_10_features['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
[0.04281151 0.70285598 0.01730094 ... 0.05803335 0.23866375 0.26528154]


,id,Exited
0,165034,0.042812
1,165035,0.702856
2,165036,0.017301
3,165037,0.301090
4,165038,0.360988
...,...,...
110018,275052,0.102891
110019,275053,0.137378
110020,275054,0.058033
110021,275055,0.238664


In [ ]:
submission_dict.to_csv('bankChurn_rf2.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>